In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import matplotlib.pyplot as plt
import ta
import warnings
import requests
warnings.filterwarnings("ignore")

# Scrape the S&P 500 constituents table from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
response = requests.get(url, headers=headers)
sp500 = pd.read_html(response.text)[0]

# Standardize ticker symbols and extract a unique list of S&P 500 tickers
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2025-12-15'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

# Download data
df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()

df

In [ ]:
# Calculate Garman-Klass volatility using high, low, open and adjusted close prices
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2 / 2 - (2 * np.log(2) - 1) * (np.log(df['close']) - np.log(df['open']))**2)

# Calculate rsi on each stock
df['rsi'] = df.groupby(level=1)['close'].transform(lambda x: ta.momentum.rsi(close=x, window=20))

# Calculate Bollinger Bands for all stocks
df['bb_low'] = df.groupby(level=1)['close'].transform(lambda x: ta.volatility.BollingerBands(close=np.log1p(x), window=20).bollinger_lband())

df['bb_mid'] = df.groupby(level=1)['close'].transform(lambda x: ta.volatility.BollingerBands(close=np.log1p(x), window=20).bollinger_mavg())

df['bb_high'] = df.groupby(level=1)['close'].transform(lambda x: ta.volatility.BollingerBands(close=np.log1p(x), window=20).bollinger_hband())

# Compute a z‑score normalized 14‑day ATR(Average True Range) to capture each stock’s relative volatility profile
def compute_atr(stock_data):
    atr_indicator = ta.volatility.AverageTrueRange(
        high=stock_data['high'],
        low=stock_data['low'],
        close=stock_data['close'],
        window=14
    )
    atr = atr_indicator.average_true_range()
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

# Compute a z‑score normalized MACD(Moving Average Convergence Divergence) signal to capture momentum shifts in each stock’s trend
def compute_macd(close):
    macd_indicator = ta.trend.MACD(close=close, window_slow=26, window_fast=12, window_sign=9)
    macd = macd_indicator.macd()
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['close'].apply(compute_macd)

# Calculate dollar volume (price × shares traded) to measure liquidity in millions of dollars
df['dollar_volume'] = (df['close']*df['volume']) / 1e6
df

Price                   close        high         low        open  \
date       ticker                                                   
2017-12-18 A        63.697105   64.497319   63.527648   63.913630   
           AAPL     41.316124   41.498793   40.950785   40.955470   
           ABBV     69.745361   70.391741   69.290763   69.375997   
           ABT      49.003815   49.108077   48.152327   48.239215   
           ACGL     28.961201   29.338389   28.923166   29.217944   
...                       ...         ...         ...         ...   
2025-12-12 XYZ      64.750000   65.120003   62.799999   63.610001   
           YUM     151.059998  151.169998  148.570007  148.899994   
           ZBH      91.589996   93.160004   90.570000   92.599998   
           ZBRA    268.390015  274.269989  265.970001  273.630005   
           ZTS     118.660004  121.080002  118.209999  119.190002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2017-12-18 A         2306300.0          0.000110        NaN       NaN   
           AAPL    117684400.0          0.000059        NaN       NaN   
           ABBV      4768300.0          0.000113        NaN       NaN   
           ABT       6053900.0          0.000098        NaN       NaN   
           ACGL      1212300.0          0.000071        NaN       NaN   
...                        ...               ...        ...       ...   
2025-12-12 XYZ       6642300.0          0.000536  47.124784  4.078423   
           YUM       1894500.0          0.000070  55.902247  4.964153   
           ZBH       2734200.0          0.000351  43.820179  4.484522   
           ZBRA       543200.0          0.000328  51.744682  5.418711   
           ZTS       5071100.0          0.000280  37.544963  4.738645   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2017-12-18 A            NaN       NaN -2.656965       NaN     146.904634  
           AAPL         NaN       NaN -1.819492       NaN    4862.263259  
           ABBV         NaN       NaN -2.172871       NaN     332.566806  
           ABT          NaN       NaN -2.741538       NaN     296.664194  
           ACGL         NaN       NaN -2.030007       NaN      35.109664  
...                     ...       ...       ...       ...            ...  
2025-12-12 XYZ     4.145287  4.212151 -0.669111 -0.379975     430.088925  
           YUM     5.006037  5.047920  1.343385 -0.208794     286.183165  
           ZBH     4.540548  4.596574 -0.415566 -0.266276     250.425368  
           ZBRA    5.530683  5.642655 -0.375786  0.082978     145.789456  
           ZTS     4.803912  4.869178 -0.022919 -1.382657     601.736745  

[982534 rows x 13 columns]